In [30]:
import numpy as np
from typing import List, Dict, Tuple

In [31]:
def forward_linear_regression(X_batch: np.ndarray, 
                              y_batch: np.ndarray,
                              weights: Dict[str, np.ndarray]) -> Tuple[float, Dict[str, np.ndarray]]:
    # assert batch sizes of X and y are equal
    assert X_batch.shape[0] == y_batch.shape[0]

    # assert the matrix muplication can work
    assert X_batch.shape[1] == weighs['W'].shape[0]

    # assert that B is simply a 1x1 ndarray
    assert weights['B'].shape[0] == weights['B'].shape[1] == 1

    # compute the operations on the forward pass
    N = np.dot(X_batch, weights['W'])

    P = N + weights['B']

    loss = np.mean(np.power(y_batch - P, 2))

    # save the information computed on the forward pass
    forward_info: Dict[str, ndarray] = {}
    forward_info['X'] = X_batch
    forward_info['N'] = N
    forward_info['P'] = P
    forward_info['y'] = y_batch   

    return loss, forward_info                              
                                  

In [34]:
def loss_gradients(forward_info: Dict[str, np.ndarray],
                    weights: Dict[str, np.ndarray]) -> Dict[str, np.ndarray]:
    '''
    Compute dLdW and dLdB for the step-by-step linear regression model. '''
    batch_size = forward_info['X'].shape[0]
    dLdP = -2 * (forward_info['y'] - forward_info['P'])
    dPdN = np.ones_like(forward_info['N'])
    dPdB = np.ones_like(weights['B'])
    dLdN = dLdP * dPdN
    dNdW = np.transpose(forward_info['X'], (1, 0))

    dLdW = np.dot(dNdW, dLdN)
    # need to sum along dimension representing the batch size # (see note near the end of this chapter)
    dLdB = (dLdP * dPdB).sum(axis=0)
        
    loss_gradients: Dict[str, ndarray] = {}
    loss_gradients['W'] = dLdW
    loss_gradients['B'] = dLdB
    return loss_gradients

In [6]:
a.shape[0]

2

In [7]:
print(a)

[[2 3 4]
 [5 6 7]]


In [17]:
type(a)

numpy.ndarray